# Proyecto End to End de Machine Learning 
### Viviendas en venta en Madrid


## 0. Librerías
 

In [1]:
# importación agrupada de librerías necesarias en este notebook
import pandas as pd
import numpy as np
import json

import sys
import os

from scipy import stats
from PIL import Image
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

#warnings.filterwarnings('ignore')

# Añado el directorio padre (del que está este notebook) a sys.path
sys.path.append(os.path.abspath('../'))
from scripts.utils_agv import ini_inspec, crear_tabla_resumen, categoricas, numericas

## 4. Compresión de variables

In [2]:
# carga de los datos guardados en el anterior paso
df = pd.read_csv('../data/processed/ide_viv_limpieza0_2025-03-11.csv', index_col='propertyCode')
df.index.name = 'ID'
print (df.shape)
df.head(1)

(1747, 24)


,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,address,district,...,hasVideo,status,hasLift,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,topPlus
ID,,,,,,,,,,,,,,,,,,,,,
107526421,43,3,1095000.0,flat,146.0,True,3,2,calle de las Fuentes,Centro,...,True,good,True,7500.0,{'typology': 'flat'},"{'subtitle': 'Sol, Madrid', 'title': 'Piso en ...",True,True,False,True


In [3]:
ini_inspec(df)

=== TAMAÑO Y ESTRUCTURA DE LOS DATOS ===
Número total de registros (filas): 1747
Número de columnas: 24
Uso de memoria: 281.50 KB


=== TIPOS DE DATOS Y NOMBRES DE COLUMNAS ===
numPhotos           int64
floor              object
price             float64
propertyType       object
size              float64
exterior           object
rooms               int64
bathrooms           int64
address            object
district           object
neighborhood       object
latitude          float64
longitude         float64
description        object
hasVideo             bool
status             object
hasLift            object
priceByArea       float64
detailedType       object
suggestedTexts     object
hasPlan              bool
has3DTour            bool
has360               bool
topPlus              bool
dtype: object


Información detallada del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1747 entries, 107526421 to 107518858
Data columns (total 24 columns):
 #   Column          Non-Null C

Después de esta primera inspección, confirmado que no hay duplicados, vamos a abordar los problemas que observo: valores faltantes, contenidos como diccionario.

### Tratamiento (desdoblado) de las columnas cuyos valores son diccionarios

In [4]:
df['detailedType'].unique()

array(["{'typology': 'flat'}",
       "{'typology': 'flat', 'subTypology': 'studio'}",
       "{'typology': 'flat', 'subTypology': 'penthouse'}",
       "{'typology': 'flat', 'subTypology': 'duplex'}"], dtype=object)

In [5]:
df['suggestedTexts'].unique()

array(["{'subtitle': 'Sol, Madrid', 'title': 'Piso en calle de las Fuentes'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle del Divino Pastor'}",
       "{'subtitle': 'Lavapiés-Embajadores, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Sol, Madrid', 'title': 'Piso en Carrera de San Jerónimo'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Estudio en calle de San Andrés'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Ático en calle de Fuencarral'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle de Fuencarral'}",
       "{'subtitle': 'Palacio, Madrid', 'title': 'Dúplex en calle de Santiago'}",
       "{'subtitle': 'Chueca-Justicia, Madrid', 'title': 'Piso en calle de Apodaca'}",
       "{'subtitle': 'Sol, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle del Espíritu Santo'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 't

Voy a separar en columnas aquellas cuyos datos son dicionarios:


In [6]:
#FUNCIÓN PARA EXPANDIR CELDAS CON CONTENIDO DICCIONARIOS
def expand_dict_columns(df):
    """
    Expande las columnas del dataframe de Idealista que contienen diccionarios.
    
    Parámetros:
    df (pandas.DataFrame): DataFrame con datos de Idealista
    
    Retorna:
    pandas.DataFrame: DataFrame con las columnas expandidas
    """
    # Hacer una copia del dataframe original para no modificarlo
    df_processed = df.copy()
    
    def parse_dict_safely(value):
        """Convierte strings a diccionarios de forma segura sin usar ast"""
        if pd.isna(value):
            return {}
        if isinstance(value, dict):
            return value
        if isinstance(value, str) and value.strip():
            try:
                # Intentar convertir usando json.loads
                return json.loads(value)
            except json.JSONDecodeError:
                try:
                    # Si falla, corregimos comillas simples a dobles
                    value = value.replace("'", "\"")
                    return json.loads(value)
                except json.JSONDecodeError:
                    # Si aún falla, retornar vacío
                    return {}
        return {}
    
    def process_column(column_name, field_mappings):
        """
        Procesa una columna de diccionario y extrae campos específicos.
        
        Parámetros:
        column_name (str): Nombre de la columna a procesar
        field_mappings (dict): Diccionario donde la clave es el nombre del campo 
                               a extraer y el valor es un valor por defecto
        """
        if column_name not in df_processed.columns:
            return
            
        # Convertir strings a diccionarios
        df_processed[column_name] = df_processed[column_name].apply(parse_dict_safely)
        
        # Extraer cada campo del diccionario
        for field, default_value in field_mappings.items():
            new_column_name = f"{column_name}_{field}"
            df_processed[new_column_name] = df_processed[column_name].apply(
                lambda x: x.get(field, default_value) if isinstance(x, dict) else default_value
            )
    
    # Definir los campos a extraer para cada columna
    column_fields = {
        'suggestedTexts': {'subtitle': None, 'title': None},
        'detailedType': {'typology': None, 'subTypology': None},
        'parkingSpace': {
            'hasParkingSpace': False, 
            'isParkingSpaceIncludedInPrice': None,
            'parkingSpacePrice': None
        }
    }
    
    # Procesar cada columna
    for column, fields in column_fields.items():
        process_column(column, fields)
    
    # Eliminar las columnas originales
    columns_to_drop = [col for col in column_fields.keys() if col in df_processed.columns]
    df_processed = df_processed.drop(columns=columns_to_drop)
    
    return df_processed

In [7]:
# CELDA SALTADA intento de tratamiento de todas las columnas con diccionarios, incluso Parking
pass
def expand_dict_columns(df):
    """
    Expande las columnas del dataframe de Idealista que contienen diccionarios.
    
    Parámetros:
    df (pandas.DataFrame): DataFrame con datos de Idealista
    
    Retorna:
    pandas.DataFrame: DataFrame con las columnas expandidas
    """
    # Hacer una copia del dataframe original para no modificarlo
    df_processed = df.copy()
    
    def parse_dict_safely(value):
        """Convierte strings a diccionarios de forma segura sin usar ast"""
        if pd.isna(value):
            return {}
        if isinstance(value, dict):
            return value
        if isinstance(value, str) and value.strip():
            try:
                # Intentar convertir usando json.loads
                return json.loads(value)
            except json.JSONDecodeError:
                try:
                    # Si falla, corregimos comillas simples a dobles
                    value = value.replace("'", "\"")
                    return json.loads(value)
                except json.JSONDecodeError:
                    # Si aún falla, retornar vacío
                    return {}
        return {}
    
    # Procesar columna suggestedTexts
    if 'suggestedTexts' in df_processed.columns:
        df_processed['suggestedTexts'] = df_processed['suggestedTexts'].apply(parse_dict_safely)
        df_processed['suggestedTexts_subtitle'] = df_processed['suggestedTexts'].apply(
            lambda x: x.get('subtitle') if isinstance(x, dict) else None
        )
        df_processed['suggestedTexts_title'] = df_processed['suggestedTexts'].apply(
            lambda x: x.get('title') if isinstance(x, dict) else None
        )
    
    # Procesar columna detailedType
    if 'detailedType' in df_processed.columns:
        df_processed['detailedType'] = df_processed['detailedType'].apply(parse_dict_safely)
        df_processed['detailedType_typology'] = df_processed['detailedType'].apply(
            lambda x: x.get('typology') if isinstance(x, dict) else None
        )
        df_processed['detailedType_subTypology'] = df_processed['detailedType'].apply(
            lambda x: x.get('subTypology') if isinstance(x, dict) else None
        )
    
    # Procesar columna parkingSpace - Esto lo dejamos explícito para manejar mejor los casos especiales
    if 'parkingSpace' in df_processed.columns:
        # Convertir strings a diccionarios y manejar valores NaN
        df_processed['parkingSpace'] = df_processed['parkingSpace'].apply(parse_dict_safely)
        
        # Extraer hasParkingSpace - valor por defecto es False
        df_processed['parkingSpace_hasParkingSpace'] = df_processed['parkingSpace'].apply(
            lambda x: x.get('hasParkingSpace', False) if isinstance(x, dict) else False
        )
        
        # Extraer isParkingSpaceIncludedInPrice - Sin valor por defecto para preservar NaN cuando no existe
        df_processed['parkingSpace_isParkingSpaceIncludedInPrice'] = df_processed['parkingSpace'].apply(
            lambda x: x.get('isParkingSpaceIncludedInPrice') if isinstance(x, dict) else pd.NA
        )
        
        # Extraer parkingSpacePrice - Convertimos a float explícitamente si existe
        df_processed['parkingSpace_parkingSpacePrice'] = df_processed['parkingSpace'].apply(
            lambda x: float(x.get('parkingSpacePrice')) if isinstance(x, dict) and 'parkingSpacePrice' in x and x['parkingSpacePrice'] is not None else pd.NA
        )
    
    # Eliminar las columnas originales que contenían diccionarios
    columns_to_drop = []
    for col in ['suggestedTexts', 'detailedType', 'parkingSpace']:
        if col in df_processed.columns:
            columns_to_drop.append(col)
    
    df_processed = df_processed.drop(columns=columns_to_drop)
    return df_processed

In [8]:
# df_ampliado_dict = expand_dict_columns(df, dict_columns=['otraColumnaDict', 'segundaColumnaDict'])
df_ampliado_dict = expand_dict_columns(df)
df_ampliado_dict

,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,address,district,...,hasLift,priceByArea,hasPlan,has3DTour,has360,topPlus,suggestedTexts_subtitle,suggestedTexts_title,detailedType_typology,detailedType_subTypology
ID,,,,,,,,,,,,,,,,,,,,,
107526421,43,3,1095000.0,flat,146.0,True,3,2,calle de las Fuentes,Centro,...,True,7500.0,True,True,False,True,"Sol, Madrid",Piso en calle de las Fuentes,flat,None
107246640,27,2,1195000.0,flat,134.0,True,3,3,calle del Divino Pastor,Centro,...,True,8918.0,True,True,True,True,"Malasaña-Universidad, Madrid",Piso en calle del Divino Pastor,flat,None
106773918,33,3,790000.0,flat,148.0,True,2,2,Barrio Lavapiés-Embajadores,Centro,...,True,5338.0,True,True,False,True,"Lavapiés-Embajadores, Madrid",Piso,flat,None
102711209,44,5,2350000.0,flat,218.0,True,4,4,Carrera de San Jerónimo,Centro,...,True,10780.0,True,True,False,True,"Sol, Madrid",Piso en Carrera de San Jerónimo,flat,None
106822977,34,bj,235000.0,studio,38.0,False,0,1,calle de San Andrés,Centro,...,True,6184.0,True,True,False,True,"Malasaña-Universidad, Madrid",Estudio en calle de San Andrés,flat,studio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107419060,0,3,4200000.0,flat,288.0,True,3,3,calle del Almirante,Centro,...,True,14583.0,False,False,False,False,"Chueca-Justicia, Madrid",Piso en calle del Almirante,flat,None
106976644,0,1,2810000.0,flat,238.0,True,3,4,Barrio Sol,Centro,...,True,11807.0,False,False,False,False,"Sol, Madrid",Piso,flat,None
107230033,0,4,2170000.0,penthouse,125.0,True,2,3,Barrio Chueca-Justicia,Centro,...,True,17360.0,False,False,False,False,"Chueca-Justicia, Madrid",Ático,flat,penthouse


In [9]:
# conversión Parking Space en multiples columnas bien: finalmente esto no lo he realizado, aunque hice multiples intentos, y nunca lograba que me funcionara bien
# TODO mejorar esto
# df_ampliado_dict['parkingSpace_hasParkingSpace'].unique()
# df_ampliado_dict['parkingSpace_isParkingSpaceIncludedInPrice'].unique()
# df_ampliado_dict['parkingSpace_parkingSpacePrice'].unique()

In [10]:
df2=df_ampliado_dict.copy()

In [11]:
df2['detailedType_typology'].unique()

array(['flat'], dtype=object)

In [12]:
df2['detailedType_subTypology'].unique()

array([None, 'studio', 'penthouse', 'duplex'], dtype=object)

In [13]:
ini_inspec(df2)

=== TAMAÑO Y ESTRUCTURA DE LOS DATOS ===
Número total de registros (filas): 1747
Número de columnas: 26
Uso de memoria: 308.80 KB


=== TIPOS DE DATOS Y NOMBRES DE COLUMNAS ===
numPhotos                     int64
floor                        object
price                       float64
propertyType                 object
size                        float64
exterior                     object
rooms                         int64
bathrooms                     int64
address                      object
district                     object
neighborhood                 object
latitude                    float64
longitude                   float64
description                  object
hasVideo                       bool
status                       object
hasLift                      object
priceByArea                 float64
hasPlan                        bool
has3DTour                      bool
has360                         bool
topPlus                        bool
suggestedTexts_subtitle      ob

### Análisis de las variables

A continuación, una rápida analítica de cada una de las variables.
1. **Variable**: nombre variable/alias
2. **Data type**: cualitativa, cuantitativa, ordinal, continua...¿?
3. **Segmento**: clasificar las variables según su significado. Si son variables demográficas, económicas, identificadores, tiempo...
4. **Expectativas**: un pequeño indicador personal de si resultará útil la variable. ¿Necesito esta variable para la solución? ¿Cómo de importante será esta variable? ¿Esta info la recoge otra variable ya vista?
5. **Conclusiones**: después del análisis anterior, llegar a unas conclusiones sobre la importancia de la variable.

|Variable |Dtype |tipo |faltantes |segmento |expectativas |conclusiones|
|--|--|--|--|--|--|--|
|||||unidades |descripción |
|propertyCode(ID)| int64| entero|código numérico Idealista||ID
|numPhotos|int64 | entero||ud.|posible categorizador|probar| 
|floor |object | entero| 65|piso|valiosa|importante|
|price | float64| continuo||€|predicción|target|
|propertyType| object |categórico|
|size| float64| continuo|
|exterior| object| booleano |24|bool|valioso|importante|
|rooms| int64| entero|||||
|bathrooms| int64| entero|
|address| object| categórico|
|district| object| categórico|
|neighborhood| object| categórico|
|latitude|float64 | continuo|
|longitude|float64 | continuo|
|description| object| categórico|
|hasVideo| bool| booleano |
|status| object| categórico|5|valoración incremental discreta|valiosa|ordinal encoder|
|hasLift|  object| booleano |6| bool|valiosa|importante|
|priceByArea| float64 | continuo|
|hasPlan| bool| booleano |
|has3DTour|bool| booleano |
|has360| bool| booleano |
|topPlus| bool| booleano |
|suggestedTexts_subtitle|object| categórico| | ¿datos adicionales?|¿redundante?||
|suggestedTexts_title |object| categórico| | ¿datos adicionales?|¿redundante?||
|detailedType_typology |   object| categórico| | clasificación|¿redundante?||
|detailedType_subTypology | object| categórico|1524| clasificación|¿redundante?||

### Tratamiento de la columna 'floor'

In [14]:
df2['floor'].unique()

array(['3', '2', '5', 'bj', '4', '6', '10', '1', '8', 'en', nan, '11',
       'st', '9', 'ss', '-1', '7'], dtype=object)

In [15]:
print(f"Número de valores faltantes en 'floor': {df['floor'].isnull().sum()}")
print("Filas con valores faltantes en 'floor':")
df[df['floor'].isnull()].head().T


Número de valores faltantes en 'floor': 64
Filas con valores faltantes en 'floor':


ID,105028115,106588391,106970345,107150334,107123975
numPhotos,33,49,6,43,17
floor,NaN,NaN,NaN,NaN,NaN
price,1300000.0,780000.0,1500000.0,1175000.0,900000.0
propertyType,flat,flat,flat,flat,flat
size,230.0,125.0,150.0,169.0,94.0
exterior,True,True,True,True,True
rooms,3,2,2,3,3
bathrooms,2,2,3,2,2
address,Barrio Lavapiés-Embajadores,plaza de la Cruz Verde,calle del Molino de Viento,calle de San Bernardo,calle de los Reyes
district,Centro,Centro,Centro,Centro,Centro


In [16]:
# Ajustar pandas para que no trunque el texto
pd.set_option('display.max_colwidth', None)

# Filtrar el DataFrame para obtener las filas donde 'floor' tiene valores NaN
df[df['floor'].isnull()][['floor','description']]




,floor,description
ID,,
105028115,NaN,"Distinguida vivienda en Madrid histórico Engel & Völkers presenta esta distinguida propiedad en una tranquila calle junto a la Plaza Mayor de 230 metros cuadrados, incluyendo los elementos comunes, según catastro. La propiedad destaca por su distribución y reforma, que combina con gran estilo elementos actuales con los propios de principios del siglo XX. Se encuentra lista para entrar a vivir. Se distribuye en una representativa entrada y una amplia social donde encontramos un gran salón comedor con tres balcones a la calle. La cocina, práctica y funcional, se encuentra equipada con electrodomésticos de alta gama, y zona de lavandería independiente. La zona de noche se compone de tres dormitorios, uno de ellos con vestidor y cuarto baño incorporado. Los otros dos dormitorios, también dobles, comparten el otro cuarto de baño completo. Dispone de aire acondicionado por conductos y sistema de calefacción individual. La vivienda ofrece luz natural en todas sus estancias y un gran encanto a través de su distribución ideal entre la zona social y la zona más privada. Este inmueble se encuentra en la primera planta de una finca señorial y tradicional propia de la zona de 1905, completamente restaurada, y con conserje de lunes a viernes. La finca cuenta con ascensor y elementos arquitectónicos decorativos originales en las zonas comunes. Por su cercanía al Ministerio de Asuntos Exteriores goza de gran seguridad y vigilancia en el exterior. Si se necesita se puede adquirir plaza de garaje en la finca contigua. Se trata de una propiedad exclusiva en un edificio clásico histórico en el corazón de Madrid. Por sus dimensiones y ubicación, la vivienda es apta tanto para aquéllos que quieran distinarla al alquiler de alto standing, como para los que decidan instalarse con su familia en una vivienda singular en pleno centro de Madrid. La vivienda se encuentra en una calle en pleno corazón del Madrid histórico, monumental y cultural, junto a la Puerta del Sol. La zona dispone de la más variada oferta de pequeños comercios tradicionales y cosmopolitas de Madrid. Al salir de casa, y sin necesidad de utilizar el transporte público o privado, encontramos toda la riqueza gastronómica representada por sus mejores restaurantes y bares tradicionales, así como toda la oferta cultural de la ciudad. Su proximidad a la Plaza Mayor y Puerta del Sol nos permite en un agradable paseo llegar a la zona comercial más importante del país."
106588391,NaN,"Bienvenido a este encantador apartamento en Palacio, Centro, Madrid. Con 125 m², esta propiedad ofrece un espacio generoso y bien distribuido. Cuenta con 2 dormitorios y 2 baños, ideal para una familia o para compartir. El apartamento se encuentra en buen estado y forma parte de un edificio clásico que cuenta con ascensor, combinando el encanto tradicional con la comodidad moderna. Disfrute de los días soleados en su balcón privado y manténgase cómodo durante todo el año, gracias al aire acondicionado en verano y la calefacción eléctrica o de gas en invierno. Este apartamento es una oportunidad única para vivir en pleno centro de Madrid, rodeado de historia, cultura y todos los servicios que la ciudad ofrece. Palacio es un barrio histórico y emblemático situado en el distrito Centro de Madrid. Hogar de monumentos icónicos como el Palacio Real, la Catedral de la Almudena y la Plaza de Oriente, combina historia, cultura y un ambiente único. Sus calles adoquinadas, como las de la Plaza de la Villa o el Viaducto de Segovia, ofrecen un viaje al pasado, mientras que su oferta de restaurantes, teatros y tiendas modernas garantiza una experiencia vibrante. Con excelentes conexiones de transporte público y cercanía a zonas como La Latina y Ópera, Palacio es ideal para quienes buscan vivir en un entorno cultural privilegiado en el corazón de Madrid. Perfecto para quienes buscan comodidad, estilo y una ubicación inmejorable. ¡No dejes pasar la oportunidad de convertir este espacio en t

### Tratamiento de la columna 'exterior'

In [17]:
df2['exterior'].unique()

array([True, False, nan], dtype=object)

### Tratamiento de la columna 'status'

In [18]:
df2['status'].unique()

array(['good', 'renew', 'newdevelopment', nan], dtype=object)

### Tratamiento de la columna 'lift'

In [19]:
df2['hasLift'].unique()

array([True, False, nan], dtype=object)

In [20]:
# Ajustar pandas para que no trunque el texto
pd.set_option('display.max_colwidth', None)

# Filtrar el DataFrame para obtener las filas donde 'lift' tiene valores NaN
df2[df2['hasLift'].isnull()][['hasLift','floor','description']]

,hasLift,floor,description
ID,,,
107520638,NaN,3,"CON TODO EL SABOR DE MADRID. PRÁCTICAMENTE A ESTRENAR, COQUETA VIVIENDA EN TERCERA PLANTA DE DOS DORMITORIOS DOBLES Y DOS AMPLIOS CUARTOS DE BAÑO. Exquisito piso exterior TOTALMENTE REFORMADO en la exclusiva zona de Palacio, una de las zonas mas emblemáticas de Madrid. Finca representativa y rehabilitada. El PISO Piso EXTERIOR COMPLETAMENTE REFORMADO CON ORIENTACIÓN SUR-ESTE de dos dormitorios, despacho, vestidor y dos baños, uno en suite. El salón-comedor con mas de cuarenta metros se convierte en el corazón de la vivienda. La cocina con todos los electrodomésticos (Nevera, lavavajillas, extractor decorativo, horno y microondas). Espacioso cuarto en office oculto para lavadora, secadora y aperos. Agua caliente por gas natural, calefacción individual, aire acondicionado por conducto en todas las estancias. Dispone de dos amplios dormitorios, uno en suite con amplio vestidor y despacho en estudio biblioteca, el otro doble con gran armario empotrado. Ventanas con rotura de puente térmico y acústico climalit. Suelos de madera maciza. Altas calidades. Conserje en finca representativa completamente rehabilitada. Exenta de IBI. Gastos de comunidad: 240€ LA ZONA La Calle Mayor de Madrid es una de las vías más emblemáticas y apasionantes de la ciudad, cargada de historia, anécdotas y tradición. Animado punto de encuentro tanto para madrileños como para turistas, es una de las arterias principales de la ciudad desde su fundación. Su historia se remonta al Madrid medieval, cuando la ciudad aún era una pequeña urbe amurallada. Durante siglos, fue la vía que conectaba el centro político y religioso, el Alcázar Real y la Iglesia de Santa María, con las salidas a la ciudad y el campo. Se extiende desde la Puerta del Sol hasta el Parque del Emir Mohamed I, donde empieza la Cuesta de la Vega, formando parte del casco antiguo de la ciudad y en su recorrido se encuentran algunos de los monumentos más importantes del centro histórico. A lo largo de sus 500 metros, es posible descubrir palacios, iglesias, edificios modernistas y rincones que maravillan y cuentan la historia de Madrid, desde sus orígenes medievales hasta su modernización en el siglo XX. Este es el sitio en que se encuentra este piso alto con orientación sur este. Finca rehabilitada. Consta de Salón comedor. Dos dormitorios dobles, y dos baños uno de ellos en suite. Estudio biblioteca. Cocina independiente totalmente equipada y pequeño office. Estar en Madrid, vivir Madrid."
107369701,NaN,1,"Referencia: 340VE6REG81 ¡PISO EN VENTA EN EL CORAZÓN DE MADRID! Estupendo y reformado piso en venta, ubicado en el famoso barrio de Sol, en el distrito Centro de Madrid, a pocos pasos y con vistas a la Plaza Mayor. Una oportunidad única para vivir en una de las zonas más emblemáticas de la ciudad. Este luminoso piso se encuentra listo para entrar a vivir. Cuenta con 2 zonas de dormitorio, un baño con ducha y una cocina office totalmente equipada con horno, vitrocerámica, lavavajillas, microondas. Además, dispone de aire acondicionado, calefacción con radiadores eléctricos, lavadora, interfono y ha sido recintemente reformado con muy buen gusto, ofreciendo un ambiente acogedor y moderno. Su ubicación es inmejorable, en el barrio de Sol, rodeado de todos los servicios esenciales: supermercados, restaurantes, tiendas, farmacias y zonas verdes. Además, está cerca de los principales puntos de interés de la ciudad, como la Plaza Mayor, Puerta del Sol, Gran Vía y el Palacio Real. Perfectamente comunicado con transporte público, con las estaciones de metro Sol (L1-2-3) y Ópera (L2-5-R), además de varias líneas de autobús. ¡No dejes pasar esta oportunidad de vivir en pleno centro de Madrid! Para más información o concertar una visita, no dudes en contactarnos. '."
107531019,NaN,4,"DOMUM REALTY Vende: ¡Precioso piso completamente reformado, en pleno centro de Madrid! ***EXCELENTE OPORTUNIDAD PARA INVERSORES*** ¡Para más información o para concertar su vis

In [ ]:
# Filtrar filas donde 'hasLift' es NaN
filtro = df2[df2['hasLift'].isnull()]

# Listas de expresiones a buscar
expresiones_positivas = ['tiene ascensor', 'dispone de ascensor', 'con ascensor']
expresiones_negativas = ['no tiene ascensor', 'no dispone de ascensor', 'sin ascensor']

# Iterar sobre el filtro para verificar y reemplazar en el DataFrame original
for index, row in filtro.iterrows():
    descripcion = str(row['description']).lower()
    if any(neg in descripcion for neg in expresiones_negativas):
        # Si hay una expresión negativa, reemplazar NaN por False
        df2.at[index, 'hasLift'] = False
    elif any(pos in descripcion for pos in expresiones_positivas):
        # Si hay una expresión positiva, reemplazar NaN por True
        df2.at[index, 'hasLift'] = True
    else:
        # Si no hay ninguna coincidencia, reemplazar NaN por False
        df2.at[index, 'hasLift'] = False

In [23]:
df2['hasLift'].unique()

array([True, False], dtype=object)

Realizo una función para imputar ascensor:

In [21]:
def imputar_ascensor(dataframe):
    """
    Esta función recorre un DataFrame y realiza imputaciones en la columna 'hasLift' basándose
    en las expresiones encontradas en la columna 'description'.
    
    - Si la descripción contiene expresiones positivas, reemplaza NaN en 'hasLift' por True.
    - Si la descripción contiene expresiones negativas, reemplaza NaN en 'hasLift' por False.
    - Si no se encuentra ninguna coincidencia, también reemplaza NaN por False.
    
    :param dataframe: DataFrame que debe contener las columnas 'hasLift' y 'description'.
    :return: DataFrame con los valores imputados en 'hasLift'.
    """
    # Listas de expresiones a buscar
    expresiones_positivas = ['tiene ascensor', 'dispone de ascensor', 'con ascensor']
    expresiones_negativas = ['no tiene ascensor', 'no dispone de ascensor', 'sin ascensor']

    # Filtrar las filas donde 'hasLift' es NaN
    filtro = dataframe[dataframe['hasLift'].isnull()]

    # Iterar sobre el filtro para verificar y reemplazar en el DataFrame original
    for index, row in filtro.iterrows():
        descripcion = str(row['description']).lower()
        if any(neg in descripcion for neg in expresiones_negativas):
            # Si hay una expresión negativa, reemplazar NaN por False
            dataframe.at[index, 'hasLift'] = False
        elif any(pos in descripcion for pos in expresiones_positivas):
            # Si hay una expresión positiva, reemplazar NaN por True
            dataframe.at[index, 'hasLift'] = True
        else:
            # Si no hay coincidencias, reemplazar NaN por False
            dataframe.at[index, 'hasLift'] = False

    return dataframe


In [22]:
imputar_ascensor(df2)

,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,address,district,...,hasLift,priceByArea,hasPlan,has3DTour,has360,topPlus,suggestedTexts_subtitle,suggestedTexts_title,detailedType_typology,detailedType_subTypology
ID,,,,,,,,,,,,,,,,,,,,,
107526421,43,3,1095000.0,flat,146.0,True,3,2,calle de las Fuentes,Centro,...,True,7500.0,True,True,False,True,"Sol, Madrid",Piso en calle de las Fuentes,flat,None
107246640,27,2,1195000.0,flat,134.0,True,3,3,calle del Divino Pastor,Centro,...,True,8918.0,True,True,True,True,"Malasaña-Universidad, Madrid",Piso en calle del Divino Pastor,flat,None
106773918,33,3,790000.0,flat,148.0,True,2,2,Barrio Lavapiés-Embajadores,Centro,...,True,5338.0,True,True,False,True,"Lavapiés-Embajadores, Madrid",Piso,flat,None
102711209,44,5,2350000.0,flat,218.0,True,4,4,Carrera de San Jerónimo,Centro,...,True,10780.0,True,True,False,True,"Sol, Madrid",Piso en Carrera de San Jerónimo,flat,None
106822977,34,bj,235000.0,studio,38.0,False,0,1,calle de San Andrés,Centro,...,True,6184.0,True,True,False,True,"Malasaña-Universidad, Madrid",Estudio en calle de San Andrés,flat,studio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107419060,0,3,4200000.0,flat,288.0,True,3,3,calle del Almirante,Centro,...,True,14583.0,False,False,False,False,"Chueca-Justicia, Madrid",Piso en calle del Almirante,flat,None
106976644,0,1,2810000.0,flat,238.0,True,3,4,Barrio Sol,Centro,...,True,11807.0,False,False,False,False,"Sol, Madrid",Piso,flat,None
107230033,0,4,2170000.0,penthouse,125.0,True,2,3,Barrio Chueca-Justicia,Centro,...,True,17360.0,False,False,False,False,"Chueca-Justicia, Madrid",Ático,flat,penthouse


### Tratamiento de la columna 'detailedType_subTypology'

## 15. División en train y test
(resevar una porción de los datos obtenidos para probar nuestros modelos)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'],
                                                    test_size=0.2,
                                                    random_state=42)

___________________________________________
## XX. Transformaciones a aplicar a test (nuevos datos que vengan)

In [ ]:
#agrupación del código a aplicar a test, que apliqué en train

# Detener la ejecución de esta celda

# Todo lo que esté aquí no se ejecutará
print("Esta celda está reservada para aplicar a los datos de test.")
return

# # Definir la columna target
target = 'price'  # Reemplaza con el nombre real de la columna target

# Verificar si la columna target está en el DataFrame y eliminarla si existe
if target in datos_test.columns:
    datos_test = datos_test.drop(columns=[target])

# Continuar con el resto del código sin interrupciones
# df.drop_duplicates(keep='first', inplace=True)    #no es necesario eliminar duplicados en el test

df = datos_test.reset_index(drop=True).set_index("propertyCode")
df.index.name = 'ID'

col_eliminar = ['thumbnail','externalReference', 'priceInfo', 'operation', 'province', 'municipality',
       'country', 'showAddress', 'url', 'newDevelopment', 'change', 'highlight', 'savedAd', 
       'notes','hasStaging', 'topNewDevelopment', 'parkingSpace' ,'newDevelopmentFinished' ]
df.drop(col_eliminar, axis=1, inplace=True)

df = expand_dict_columns(df)
